In [14]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

#### Read Power grid info

In [15]:
# Create buses
num_buses = 118
buses = np.arange(1, num_buses+1)

# Read gen buses
file_path = '../../data/IEEE_Case118/zones/gen_bus.csv'
gen_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read thermal buses
file_path = '../../data/IEEE_Case118/zones/thermal_bus.csv'
thermal_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read wind gen buses
file_path = '../../data/IEEE_Case118/zones/wind_bus.csv'
wind_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read load buses
file_path = '../../data/IEEE_Case118/zones/load_bus.csv'
load_buses = pd.read_csv(file_path, header=None, index_col=None)

In [16]:
# Boolean index of thermal buses
thermal_bidx = np.isin(buses, thermal_buses)

# Boolean index of wind buses
wind_bidx = np.isin(buses, wind_buses)

# Boolean index of load buses
load_bidx = np.isin(buses, load_buses)

In [17]:
# Read zone 1 thermal load buses
file_path = f'../../data/IEEE_Case118/zones/zone1_thermal_buses.csv'
zone1_thermal_buses = pd.read_csv(file_path, header=None, index_col=None)
file_path = f'../../data/IEEE_Case118/zones/zone1_load_bus.csv'
zone1_load_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read zone 2 thermal load buses
file_path = f'../../data/IEEE_Case118/zones/zone2_thermal_buses.csv'
zone2_thermal_buses = pd.read_csv(file_path, header=None, index_col=None)
file_path = f'../../data/IEEE_Case118/zones/zone2_load_bus.csv'
zone2_load_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read zone 3 thermal load buses
file_path = f'../../data/IEEE_Case118/zones/zone3_thermal_buses.csv'
zone3_thermal_buses = pd.read_csv(file_path, header=None, index_col=None)
file_path = f'../../data/IEEE_Case118/zones/zone3_load_bus.csv'
zone3_load_buses = pd.read_csv(file_path, header=None, index_col=None)

In [18]:
# Get zone1 thermal and load Boolean index
zone1_thermal_bidx = np.isin(buses, zone1_thermal_buses)
zone1_load_bidx = np.isin(buses, zone1_load_buses)

# Get zone2 thermal and load Boolean index
zone2_thermal_bidx = np.isin(buses, zone2_thermal_buses)
zone2_load_bidx = np.isin(buses, zone2_load_buses)

# Get zone1 thermal and load Boolean index
zone3_thermal_bidx = np.isin(buses, zone3_thermal_buses)
zone3_load_bidx = np.isin(buses, zone3_load_buses)

#### Create node features

In [19]:
nt = 12     # Time steps

num_zones = 3  # Number of zones

num_samples = 1000  # Number of SCUC samples
max_num = 1199  # Max number

# Empty list to store data
x = []

# Assign wind and load features
node_features = np.zeros((num_buses, 2*nt))

for i in range(max_num):
    # File path
    wind_path = f'../../data/IEEE_Case118/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case118/inputs/load/sample_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, :nt] = wind.values
    node_features[load_bidx, -nt:] = load.values

    x.append(node_features.copy())

The file sample_4 is not found
The file sample_8 is not found
The file sample_15 is not found
The file sample_25 is not found
The file sample_27 is not found
The file sample_35 is not found
The file sample_37 is not found
The file sample_41 is not found
The file sample_42 is not found
The file sample_43 is not found
The file sample_45 is not found
The file sample_57 is not found
The file sample_58 is not found
The file sample_64 is not found
The file sample_71 is not found
The file sample_80 is not found
The file sample_81 is not found
The file sample_85 is not found
The file sample_86 is not found
The file sample_89 is not found
The file sample_91 is not found
The file sample_97 is not found
The file sample_99 is not found
The file sample_102 is not found
The file sample_114 is not found
The file sample_124 is not found
The file sample_144 is not found
The file sample_149 is not found
The file sample_152 is not found
The file sample_158 is not found
The file sample_175 is not found
Th

The file sample_214 is not found
The file sample_224 is not found
The file sample_225 is not found
The file sample_240 is not found
The file sample_253 is not found
The file sample_263 is not found
The file sample_273 is not found
The file sample_274 is not found
The file sample_285 is not found
The file sample_287 is not found
The file sample_297 is not found
The file sample_306 is not found
The file sample_313 is not found
The file sample_324 is not found
The file sample_325 is not found
The file sample_328 is not found
The file sample_334 is not found
The file sample_335 is not found
The file sample_336 is not found
The file sample_340 is not found
The file sample_355 is not found
The file sample_360 is not found
The file sample_366 is not found
The file sample_388 is not found
The file sample_390 is not found
The file sample_398 is not found
The file sample_414 is not found
The file sample_415 is not found
The file sample_419 is not found
The file sample_420 is not found
The file s

#### Create target features

In [20]:
# List to store target features
y = []

# Target features
agg_PG = np.zeros((num_zones+1, nt))

temp = np.zeros((num_buses, nt))
temp2 = np.zeros((num_buses-num_zones-1, nt))

for i in range(max_num):
    PG_path = f'../../data/IEEE_Case118/outputs/PG/sample_{i+1}.csv'
    try:
        PG = pd.read_csv(PG_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    temp[thermal_bidx] = PG.values

    agg_PG[0, :] = temp.sum(axis=0)
    agg_PG[1, :] = temp[zone1_thermal_bidx].sum(axis=0)
    agg_PG[2, :] = temp[zone2_thermal_bidx].sum(axis=0)
    agg_PG[3, :] = temp[zone3_thermal_bidx].sum(axis=0)

    # y.append(np.concatenate((agg_PG, temp2), axis=0))
    y.append(agg_PG.copy())

The file sample_4 is not found
The file sample_8 is not found
The file sample_15 is not found
The file sample_25 is not found
The file sample_27 is not found
The file sample_35 is not found
The file sample_37 is not found
The file sample_41 is not found
The file sample_42 is not found
The file sample_43 is not found
The file sample_45 is not found
The file sample_57 is not found
The file sample_58 is not found
The file sample_64 is not found
The file sample_71 is not found
The file sample_80 is not found
The file sample_81 is not found
The file sample_85 is not found
The file sample_86 is not found
The file sample_89 is not found
The file sample_91 is not found
The file sample_97 is not found
The file sample_99 is not found
The file sample_102 is not found
The file sample_114 is not found
The file sample_124 is not found
The file sample_144 is not found
The file sample_149 is not found
The file sample_152 is not found
The file sample_158 is not found


The file sample_175 is not found
The file sample_176 is not found
The file sample_185 is not found
The file sample_207 is not found
The file sample_210 is not found
The file sample_214 is not found
The file sample_224 is not found
The file sample_225 is not found
The file sample_240 is not found
The file sample_253 is not found
The file sample_263 is not found
The file sample_273 is not found
The file sample_274 is not found
The file sample_285 is not found
The file sample_287 is not found
The file sample_297 is not found
The file sample_306 is not found
The file sample_313 is not found
The file sample_324 is not found
The file sample_325 is not found
The file sample_328 is not found
The file sample_334 is not found
The file sample_335 is not found
The file sample_336 is not found
The file sample_340 is not found
The file sample_355 is not found
The file sample_360 is not found
The file sample_366 is not found
The file sample_388 is not found
The file sample_390 is not found
The file s

#### Create edge index and attributes

In [21]:
# Read branch info
file_path = '../../data/IEEE_Case118/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Get branch index and attr
edge_index = branch[:, :2] - 1
edge_attr = branch[:, 2:]

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)
edge_attr = torch.from_numpy(edge_attr).float()

#### Create train, test and validation dataset

In [22]:
data_list = []

for i in range(len(x)):
    X = torch.from_numpy(x[i]).float()
    Y = torch.from_numpy(y[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

In [23]:
dir = f'../../train_val_test_dataset/IEEE_Case118/agg_PG/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

from CustomDataset import CustomDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case118/agg_PG'
CustomDataset(root=root, data_list=data_list)

The old dataset has been deleted!


Processing...
Done!


CustomDataset(1000)